# Regresja logistyczna
Do estymacji modeli będziemy używać pakietu scikit-learn 
https://scikit-learn.org/stable/install.html

In [ ]:

#pip install scikit-learn

In [ ]:
# Instalacja scikit-learn
#!python -m pip install scikit-learn

In [1]:
# Import wymaganych bibliotek
from  sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Zbiór danych 
https://www.kaggle.com/datasets/erdemtaha/cancer-data

Będziemy pracować na zbiorze danych dotyczącym raka piersi.

### Problem biznesowy:
Zadanie będzie polegać na zaprognozowaniu czy dana pacjentka ma raka piersi łagodnego czy złośliwego.
Rozwiązanie to powinno wspomóc lekarzy, w celu zmniejszenia umieralności.

In [ ]:
# Sprawdzenie ścieżki
os.getcwd()

In [2]:
# Katalog nadrzędny
os.chdir('../')

In [ ]:
# Ponowne sprawdzenie ścieżki
os.getcwd()

In [4]:
# puść ten kod, 
# jeżeli wywołujesz plik  w folderze rozwiąznaia, 
# a ramka danych znajduje się w folderze data
import os 
os.chdir('../')

In [5]:
# Pobranie danych

df = pd.read_csv('data/Cancer_Data.csv')

## Eksploracja danych

In [ ]:
# Nagłówek
df.head()

In [12]:
# usunięcie ostatniej zmiennej
del df['Unnamed: 32']

In [ ]:
# typy danych
df.info()

Wszystkie zmienne poza targetem (diagnosis) są numeryczne. Scikit-learn, nawet w przypadku zmiennych kategorycznych, wymaga ich przekodowania.

In [ ]:
# sprawdzenie braków danych
df.isna().max()

Chcemy modelować zjawisko wykrywania nowotworu złośliwego, więc jako 1 oznaczymy właśnie ten typ choroby.

In [ ]:
# Liczebność kategorii zmiennej celu
df['diagnosis'].value_counts()

In [14]:
# Enkodowanie zmiennej celu
df['target'] = (df['diagnosis']== 'M').astype(int)

In [ ]:
# badanie korelacji
df[df.columns[2:]].corr(method='spearman')['target'].sort_values()

W przypadku zmiennych kategorycznych należy użyć korelacji rang spearmana. Kilka zmiennych ma niską wartość korelacji, jednak jest sporo z korelacją co najmniej 0.7. Sprawdźmy czy korelacja pomiedzy zmiennymi nie jest zbyt wysoka.


In [ ]:
# Macierz korelacji
plt.figure(figsize = (17,15))
sns.heatmap(round(df[df.columns[2:]].corr(method='spearman').sort_values(by='target'),2), annot=True, linewidths=0.1)
plt.show()

In [ ]:
# Korelacja z najistotniejszą zmienną..
plt.figure(figsize = (6,8))
sns.heatmap(round(df[df.columns[2:]].corr(method='spearman').sort_values(by='target'),2)[['target','perimeter_worst']], annot=True, linewidths=0.1)
plt.show()

In [ ]:
# ...i kolejną...
plt.figure(figsize = (6,8))
sns.heatmap(round(df[df.columns[2:]].corr(method='spearman').sort_values(by='target'),2)[['target','perimeter_worst','perimeter_se']], annot=True, linewidths=0.1)
plt.show()

In [ ]:
# ...i kolejną...
plt.figure(figsize = (6,8))
sns.heatmap(round(df[df.columns[2:]].corr(method='spearman').sort_values(by='target'),2)[['target','perimeter_worst','perimeter_se','compactness_worst']], annot=True, linewidths=0.1)
plt.show()

In [ ]:
# ...i jeszcze jedną
plt.figure(figsize = (6,8))
sns.heatmap(round(df[df.columns[2:]].corr(method='spearman').sort_values(by='target'),2)[['target','perimeter_worst','perimeter_se','compactness_worst', 'concave points_se']], annot=True, linewidths=0.1)
plt.show()

In [38]:
# Finalne zmienne w modelu
x_names = ['perimeter_worst','perimeter_se','compactness_worst', 'concave points_se','texture_worst']

## Badanie outlierów
Podstawową metodą badania outlierów jest użycie rozstępu międzykwartylowego. Powinniśmy uzyskać te same wartości, które są widoczne na box-plotach. Teoria mówi, że metoda działa dobrze dla rozkładów normalnych. Często jednak nie sprawdza się tego założenia, o ile nie mamy mocno skośnego rozkładu. W powyższych zmiennych histogramy przypominają kształtem rozkład normalny.

Przypomnienie:
$BW = [Q1 − 1.5 × IQR, Q3 + 1.5 × IQR]$, gdzie $IQR = Q3 – Q1$

Outlier to wartość wykraczająca poza obszar BW. Niekiedy wartość 1.5 ustala się na wyższą


In [67]:
# Napiszemy funkcję do znajdowania outlierów
def find_outliers(x, a= 1.5):
    q1,q3 = np.quantile(x,[0.25,0.75])
    iqr = q3 - q1
    x_min = q1 - a * iqr
    x_max = q3 + a * iqr
    return (x< x_min) | (x > x_max)

In [ ]:
# Sprawdzenie funkcji
find_outliers(df.texture_mean)

In [ ]:
def find_outliers(x, a= 1.5):
    q1,q3 = np.quantile(x,[0.25,0.75])
    iqr = q3 - q1
    x_min = q1 - a * iqr
    x_max = q3 + a * iqr
    return (x< x_min) | (x > x_max)

In [69]:
# Znalezienie outlierów dla wytypowanych zmiennych
outliers_names = []
for i in x_names:
    df[f'{i}_outlier'] = find_outliers(df[i])
    outliers_names.append(f'{i}_outlier')

In [70]:
# wyznaczenie finalnej zmiennej z outlierami
df['outlier_total'] = df[outliers_names].max(axis=1)

In [ ]:
# Liczebność
df['outlier_total'].value_counts()

## Podział zbioru na trening i test
Używamy tylko jednego modelu, więc zbiór walidacyjny nie jest potrzebny. Dodatkowo, zbiór jest niewielki, więc byłoby utrudnione, aby wydzielić 3 zbiory
Do wyznaczenia zbioru treningowego i testowego użyjemy funkcji train_test_split

Dokumentacja: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
# definicja X i y
X = df.loc[~(df.outlier_total),x_names]
y = df.loc[~(df.outlier_total),'target']

In [74]:
# Funkcja zwraca 4 objekty. W argumentach używamy test_size lub train_size, który najczęsciej jest udziałem. random_state ustala ziarno losowania, aby wyniki było powtarzalne
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size=0.3, random_state=123)

In [ ]:
train_x.head()

In [77]:
# Zapisanie zbiorów na potrzeby przyszłych zajęć
train_x.to_csv('data/cancer_train_x.csv')
test_x.to_csv('data/cancer_test_x.csv')
train_y.to_csv('data/cancer_train_y.csv')
test_y.to_csv('data/cancer_test_y.csv')

## Modelowanie
Dokumentacja: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [78]:
# Stworzenie objektu modelu
model_1 = LogisticRegression()

In [ ]:
# Estymacja 
model_1.fit(train_x,train_y)

## Ocena jakości


In [80]:
# predykcje
train_pred = model_1.predict(train_x)
test_pred = model_1.predict(test_x)

In [81]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# Confusion matrix train
confusion_matrix(train_y, train_pred)

In [ ]:
# Confusion matrix test
confusion_matrix(test_y, test_pred)

In [ ]:
# Accuracy train
accuracy_score(train_y,train_pred)

In [ ]:
# Accuracy test
accuracy_score(test_y, test_pred)

Jakość modelu jest bardzo wysoka i stabilna. W kolejnym kroku należy zastanowić się nad celem biznesowym, aby wybrać najlepszą metrykę.